In [1]:
import pandas as pd

In [14]:
import pandas as pd
import json

with open('/Users/jarana/workspace/projects/Entity-Attribute-Identification/data/queries.json') as f:
    jsonstr = json.load(f)

df = pd.DataFrame(jsonstr.items(), columns=['qid', 'text'])
df['type'] = ["query"] * len(df)
df[['type', 'qid', 'text']].to_csv("/Users/jarana/workspace/WikiHow-Task-Based/data/cedr/eai-query.tsv", header=False, index=False, sep="\t")

In [51]:
df = pd.read_csv('/Users/jarana/workspace/projects/Entity-Attribute-Identification/data/qrel.txt',names=["qid", "none", "pid", "rele_label"], sep="\t")
df["pid"] = [i.split(":")[1].replace(">","") for i in df.pid.tolist()]

In [55]:
df.to_csv("/Users/jarana/workspace/WikiHow-Task-Based/data/cedr/eai-qrel.tsv", header=False, index=False, sep="\t")

In [66]:
doc = pd.DataFrame(zip(["doc"] * len(df.pid.unique()),df.pid.unique(),df.pid.unique()), columns=["type", "did", "text"])
doc.to_csv("/Users/jarana/workspace/WikiHow-Task-Based/data/cedr/eai-doc.tsv", header=False, index=False, sep="\t")

In [76]:
df.head()

,qid,none,pid,rele_label
0,INEX_LD-2009039,Q0,capital,1
1,INEX_LD-2009039,Q0,governmentType,1
2,INEX_LD-2009039,Q0,governmentType,1
3,INEX_LD-2009061,Q0,place,1
4,INEX_LD-2009063,Q0,commander,1


In [98]:
#5fold data generation
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import KFold
import random

foldNum = 5
trainId5fold = {i:[] for i in range(foldNum)}
validateId5fold = {i:[] for i in range(foldNum)}
testId5fold = {i:[] for i in range(foldNum)}

ids = df.qid.unique().tolist()
kf = KFold(n_splits=foldNum)
kf.get_n_splits(ids)
for idx, ele in enumerate(kf.split(ids)):
    train, test = [ids[i] for i in ele[0]], [ids[i] for i in ele[1]]
    train, val, _, _ = train_test_split(train, train, test_size=0.2)
    trainId5fold[idx].extend(train)
    validateId5fold[idx].extend(val)
    testId5fold[idx].extend(test)

def writeRunFile(filename, _df):
    for name, group in _df.groupby("qid"):
        for idx, row in enumerate(group.iterrows()):
            with open(filename, 'a') as fd:
                fd.write(str(row[1]['qid']) +"\tQ0\t" + str(row[1]['pid']) + "\t" + str(idx+1) + "\t" + str(row[1]['rele_label']) + "\trun\n")
                
cand_props = df.pid.unique()            
negNum = 2
for fold in range(foldNum):
    for idx, group in df[df.qid.isin(trainId5fold[fold])].groupby("qid"):
        pos = group['pid'].tolist()
        neg = list(set.difference(set(cand_props), set(pos)))
        neg = random.sample(neg, len(pos) * negNum)
        for idx, row in group.iterrows():
            with open("data/cedr/eai-train%d.tsv" % fold, 'a') as fd:
                fd.write(str(row["qid"]) + "\t"+str(row["pid"])+"\n")
        for n in neg:
            with open("data/cedr/eai-train%d.tsv" % fold, 'a') as fd:
                fd.write(str(row["qid"]) + "\t"+str(n)+"\n")
    
    for idx, group in df[df.qid.isin(validateId5fold[fold])].groupby("qid"):
        pos = group['pid'].tolist()
        neg = list(set.difference(set(cand_props), set(pos)))
        
        lines = {}
        for idx, row in group.iterrows():
            lines[str(row["qid"]) + "\tQ0\t"+str(row['pid'])+"\t0\t" +str(row['rele_label'])+"\trun\n"] = row['rele_label']

        for n in neg:
            lines[str(row["qid"]) + "\tQ0\t"+str(n)+"\t0\t0\trun\n"] = 0
        lines = [i[0] for i in sorted(lines.items(), key=lambda x: x[1], reverse=True)]
        for l in lines:
            with open("data/cedr/eai-valid%d.tsv" % fold, 'a') as fd:
                fd.write(l)
                
    for idx, group in df[df.qid.isin(testId5fold[fold])].groupby("qid"):
        pos = group['pid'].tolist()
        neg = list(set.difference(set(cand_props), set(pos)))


        lines = {}
        for idx, row in group.iterrows():
            lines[str(row["qid"]) + "\tQ0\t"+str(row['pid'])+"\t0\t" +str(row['rele_label'])+"\trun\n"] = row['rele_label']

        for n in neg:
            lines[str(row["qid"]) + "\tQ0\t"+str(n)+"\t0\t0\trun\n"] = 0
        lines = [i[0] for i in sorted(lines.items(), key=lambda x: x[1], reverse=True)]
        for l in lines:
            with open("data/cedr/eai-test%d.tsv" % fold, 'a') as fd:
                fd.write(l)
    